In [1]:
import pandas as pd
from datetime import date, timedelta
from random import randint
from time import sleep
import os
import numpy as np
import pymysql
from sqlalchemy import create_engine
import joblib
import requests
from bs4 import BeautifulSoup

In [2]:
## CONNECT TO DATABASE
username = "root"
password = "root"
host = "localhost"
port = "3307"
database = "nhl_optimizer"

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

In [3]:
query = "SELECT * FROM player_data"

df = pd.read_sql(query,engine)

## Functions

In [4]:
from datetime import date, timedelta

def fetch_game_data(year: int):
    """
    Fetches the dates of NHL games and teams involved for a given year.
    
    Parameters:
    - year: The year for which you want to fetch game data.

    Returns:
    - DataFrame with dates, visitor, and home teams.
    """
    
    url = f"https://www.hockey-reference.com/leagues/NHL_{year}_games.html"
    dfs = pd.read_html(url)
    df = dfs[0]
    
    # Convert the "Date" column to datetime.date format
    df["Date"] = pd.to_datetime(df["Date"]).dt.date

    # Extract the relevant columns
    game_data = df[["Date", "Visitor", "Home"]]

    return game_data

def filter_dates_for_week(dates, start_date):
    """
    Filters the provided dates for those that fall within the week starting at start_date.
    
    Parameters:
    - dates: List of dates.
    - start_date: The starting date of the week. 

    Returns:
    - A list of dates that fall within the desired week.
    """
    # Convert start_date to a date object if it's a string
    if isinstance(start_date, str):
        start_date = date.fromisoformat(start_date)
    
    # Calculate the end date of the week
    end_date = start_date + timedelta(days=6)

    # Filter the dates
    week_dates = [d for d in dates if start_date <= d <= end_date]
    
    return week_dates

def opponents_for_team_for_week(year, start_date):
    """
    Fetches the opponents each team faces within a specified week.
    
    Parameters:
    - year: The year for which you want to check.
    - start_date: The starting date of the week.

    Returns:
    - A dictionary with teams as keys and lists of opponents as values.
    """
    
    # Fetch game data for the year
    game_data = fetch_game_data(year)
    
    # Filter the game data for the desired week
    week_dates = filter_dates_for_week(game_data["Date"].tolist(), start_date)
    week_games = game_data[game_data["Date"].isin(week_dates)]
    
    # Prepare opponent information
    visitor_opponents = week_games.set_index("Visitor")["Home"].to_dict()
    home_opponents = week_games.set_index("Home")["Visitor"].to_dict()
    
    opponents = {}
    for team in set(week_games["Visitor"].tolist() + week_games["Home"].tolist()):
        opponents[team] = list(set([visitor_opponents.get(team, "")] + [home_opponents.get(team, "")]))

    return opponents


#-------#

def calculate_weekly_team_multipliers(year, start_date, points_df):
    """
    Calculate the weekly multipliers for teams based on their opponents for a given week.
    """
    
    # Fetch the opponents for each team for the week
    games_week = opponents_for_team_for_week(year, start_date)

    weekly_multipliers = {}
    for team, opponents in games_week.items():
        multipliers = []
        for opponent in opponents:
            multiplier_row = points_df[points_df['Team Name'] == opponent]
            if not multiplier_row.empty:
                multipliers.append(multiplier_row['multiplier'].iloc[0])
            else:
                print(f"Warning: Couldn't find multiplier for opponent {opponent}. Skipping...")

        weekly_multipliers[team] = sum(multipliers) / len(multipliers) if multipliers else 1  # Defaulting to 1 if no games

    return weekly_multipliers

# Now, you can create a new column in your players DataFrame to store the composite multiplier for the week:


def games_count_for_team_for_week(year, start_date):
    """
    Fetches the number of games each team plays within a specified week.
    
    Parameters:
    - year: The year for which you want to check.
    - start_date: The starting date of the week.

    Returns:
    - A dictionary with teams as keys and the number of games they play that week as values.
    """
    
    # Fetch game data for the year
    game_data = fetch_game_data(year)
    
    # Filter the game data for the desired week
    week_dates = filter_dates_for_week(game_data["Date"].tolist(), start_date)
    week_games = game_data[game_data["Date"].isin(week_dates)]
    
    # Count the number of games for each team
    visitor_counts = week_games["Visitor"].value_counts().to_dict()
    home_counts = week_games["Home"].value_counts().to_dict()
    
    total_counts = {}
    for team in set(list(visitor_counts.keys()) + list(home_counts.keys())):
        total_counts[team] = visitor_counts.get(team, 0) + home_counts.get(team, 0)

    return total_counts

## Team Name Dict

In [5]:
# TEAM NAMES
team_abbreviations = {
 'Anaheim Ducks':'ANA',
 'Arizona Coyotes':'ARI',
 'Boston Bruins':'BOS',
 'Buffalo Sabres':'BUF',
 'Calgary Flames':'CGY',
 'Carolina Hurricanes':'CAR',
 'Chicago Blackhawks':'CHI',
 'Colorado Avalanche':'COL',
 'Columbus Blue Jackets':'CBJ',
 'Dallas Stars':'DAL',
 'Detroit Red Wings':'DET',
 'Edmonton Oilers':'EDM',
 'Florida Panthers':'FLA',
 'Los Angeles Kings':'L.A',
 'Minnesota Wild':'MIN',
 'Montreal Canadiens':'MTL',
 'Nashville Predators':'NSH',
 'New Jersey Devils':'N.J',
 'New York Islanders':'NYI',
 'New York Rangers':'NYR',
 'Ottawa Senators':'OTT',
 'Philadelphia Flyers':'PHI',
 'Pittsburgh Penguins':'PIT',
 'San Jose Sharks':'S.J',
 'Seattle Kraken':'SEA',
 'St. Louis Blues':'STL',
 'Tampa Bay Lightning':'T.B',
 'Toronto Maple Leafs':'TOR',
 'Vancouver Canucks':'VAN',
 'Vegas Golden Knights':'VGK',
 'Washington Capitals':'WSH',
 'Winnipeg Jets':'WPG'
}

## Injuries

In [6]:
# Fetch the webpage content
url = "https://www.cbssports.com/nhl/injuries/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Lists to hold our parsed data
teams = []
injury_dataframes = []

# Iterate over each TableBaseWrapper
for wrapper in soup.find_all('div', class_='TableBaseWrapper'):
    
    # Find team name
    team_name_tag = wrapper.find('span', class_='TeamName')
    if team_name_tag:
        team_name = team_name_tag.get_text(strip=True)
        teams.append(team_name)

        # Find the injury table for the team
        table = wrapper.find('table', class_='TableBase-table')
        if table:
            # Use pandas to read the table
            df = pd.read_html(str(table))[0]
            df['Team'] = team_name  # Add a column for the team name
            injury_dataframes.append(df)

# Concatenate all dataframes
injuries_df = pd.concat(injury_dataframes, ignore_index=True)

#Clean the player name up


def clean_player_name(name):
    # Split the name using spaces to identify potential last names
    parts = name.split()

    # The last word (or the two last words in the case of names like "van Riemsdyk") is likely our surname
    surname = parts[-2] + " " + parts[-1] if len(parts) > 2 and len(parts[-2]) <= 3 else parts[-1]

    # Split by surname and get the second part
    full_name = name.split(surname, 1)[1].strip()

    return full_name if full_name else surname

injuries_df['Player'] = injuries_df['Player'].apply(clean_player_name)

## Variables

In [7]:
## VARIABLES
# Get today's date
today = date.today()

fantasy_goals = 3
fantasy_assists = 2

year = 2024
# all_dates = fetch_game_dates(year)
# week_dates = filter_dates_for_week(all_dates, today)

## Win % for SoS Calc

In [8]:
## GATHER WIN % TO USE FOR SOS CALC

dfs = pd.read_html("https://www.hockey-reference.com/leagues/NHL_2023_standings.html", index_col=0)
east_df = dfs[0].reset_index().rename(columns={'index': 'Team Name'})
west_df = dfs[1].reset_index().rename(columns={'index': 'Team Name'})

east_df = east_df.drop([0, 9], axis=0).reset_index(drop=True)
west_df = west_df.drop([0, 9], axis=0).reset_index(drop=True)
east_df['Team Name'] = east_df['Team Name'].str.replace('*', '', regex=False)
west_df['Team Name'] = west_df['Team Name'].str.replace('*', '', regex=False)


east_df = east_df[['Team Name', 'PTS%']]
west_df = west_df[['Team Name', 'PTS%']]

points_df = pd.concat([east_df, west_df], ignore_index=True)
points_df['PTS%'] = points_df['PTS%'].astype(float)


points_df['Abbreviation'] = points_df['Team Name'].map(team_abbreviations)
points_df

,Team Name,PTS%,Abbreviation
0,Boston Bruins,0.823,BOS
1,Toronto Maple Leafs,0.677,TOR
2,Tampa Bay Lightning,0.598,T.B
3,Florida Panthers,0.561,FLA
4,Buffalo Sabres,0.555,BUF
5,Ottawa Senators,0.524,OTT
6,Detroit Red Wings,0.488,DET
7,Montreal Canadiens,0.415,MTL
8,Carolina Hurricanes,0.689,CAR
9,New Jersey Devils,0.683,N.J


## Load and Process Models

In [9]:
# Load the models
loaded_goals_pipeline = joblib.load('goals_model.pkl')
loaded_assists_pipeline = joblib.load('assists_model.pkl')

# Read and process the data
season_totals = pd.read_csv('season_totals_23.csv',index_col=0)
season_totals.drop(columns=["Goals/60","Total Assists/60"], inplace=True)

# Predictions
goal_predictions = loaded_goals_pipeline.predict(season_totals)
assist_predictions = loaded_assists_pipeline.predict(season_totals)

## Build Reference DataFrame

In [10]:
# Create dataframe for reference

player_df = pd.DataFrame({
    'Team':season_totals['Team'].values,
    'Position':season_totals['Position'].values,
    'Player':season_totals['Player'].values,
    'Projected Goals/60':goal_predictions,
    'Projected Assists/60':assist_predictions,
    'Projected Goals (TOI)':(season_totals['TOI/GP']/60) * goal_predictions,
    'Projected Assists (TOI)':(season_totals['TOI/GP']/60) * assist_predictions,
})

player_df = player_df.merge(points_df, left_on='Team', right_on='Abbreviation', how='left')

player_df = player_df.round(2).sort_values(by='Projected Goals (TOI)', ascending=False)
player_df['Team'] = player_df['Team Name'].str.split(',').str[0].str.strip()

player_df = player_df[['Team','Abbreviation','Player', 'Position','Projected Goals (TOI)', 'Projected Assists (TOI)']]

In [11]:
# Cleaning the team name (remove asterisks and keep only abbreviation, if needed)
points_df['Team'] = points_df['Team Name']

# Calculating multiplier based on points percentage
points_df['multiplier'] = points_df['PTS%'].apply(lambda x: 0.5 / x)

In [12]:
weekly_multipliers = calculate_weekly_team_multipliers(year, today + timedelta(days=60), points_df)
games_this_week = games_count_for_team_for_week(year, today + timedelta(days=60))


# Map this to your players DataFrame
player_df['weekly_multiplier'] = player_df['Team'].map(weekly_multipliers)
player_df['games_this_week'] = player_df['Team'].map(games_this_week)


In [13]:
player_df['projected_goals'] = player_df['Projected Goals (TOI)'] * player_df['weekly_multiplier'] * player_df['games_this_week']
player_df['projected_assists'] = player_df['Projected Assists (TOI)'] * player_df['weekly_multiplier'] * player_df['games_this_week']

# Factoring in fantasy points value for goals and assists
player_df['proj_fantasy_pts'] = (player_df['projected_goals'] * fantasy_goals) + (player_df['projected_assists'] * fantasy_assists)

# Create simple F or D position for Sportsnet
player_df.loc[player_df['Position'] != 'D', 'Position'] = 'F'


In [14]:
# Now, merge on 'Player'
merged_df = pd.merge(player_df, injuries_df[['Player', 'Injury Status']], 
                     on='Player', 
                     how='left')

# Set a new column to indicate if a player is injured or not
merged_df['Injured'] = ~merged_df['Injury Status'].isnull()

# Set the projected fantasy points to 0 for injured players
merged_df['proj_fantasy_pts'] = np.where(merged_df['Injured'], 0, merged_df['proj_fantasy_pts'])

In [15]:
projections_df = merged_df[['Team','Injured','Player','Position','weekly_multiplier','games_this_week','proj_fantasy_pts']]


In [16]:
# projections_df.to_csv('injury_report.csv',index=False)

In [17]:
filtered_projections = projections_df[projections_df['proj_fantasy_pts'] >0]
filtered_projections

,Team,Injured,Player,Position,weekly_multiplier,games_this_week,proj_fantasy_pts
0,Edmonton Oilers,False,Connor McDavid,F,1.128693,3.0,11.952858
1,Boston Bruins,False,David Pastrnak,F,0.820280,3.0,8.071553
2,Colorado Avalanche,False,Mikko Rantanen,F,0.779112,3.0,7.432731
3,Edmonton Oilers,False,Leon Draisaitl,F,1.128693,3.0,10.835452
4,Colorado Avalanche,False,Nathan MacKinnon,F,0.779112,3.0,7.572971
...,...,...,...,...,...,...,...
706,Buffalo Sabres,False,Kale Clague,D,0.909891,3.0,0.109187
708,Washington Capitals,False,Alexander Alexeyev,D,0.891266,2.0,0.213904
710,San Jose Sharks,False,Matthew Benning,D,0.751880,3.0,1.082707
712,Edmonton Oilers,False,Vincent Desharnais,D,1.128693,3.0,0.033861
